In [5]:
install -c conda-forge keras
import pandas as pd
import numpy as np
import os
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Activation
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#The following are datasets from the S&P 500 companies for their quarterly data collected from the 10-Q form on the SEC
#They are broken up into each individual sector of the S&P, which are provided by the NYSE (there are 11, we are looking
#at just 7 right now) A lot of the columns are useless information so I dropped many columns that are not useful
#Below I will explain what each column means



#GVKEY: NYSE identification number. All companies are domestic so they are given a GVKEY to help find faster

#datadate: Date identifier for which quarter it is. Data format is the uniform however the date of the report varies
#we will have to normalize this later. Most data is reported 3/31, 6/30, 9/30, 12/31, we could just remove the data
#that does not follow this pattern (ie: Best Buy) 12/31 includes the Holiday times, which we might have to control for

#conm: Company Name, just an easier way to identify the company for the human eye rather than the GVKEY. computer
#prefers GVKEY because it's numerical

#atq: Assets total quaterly (in millions), total assets of the company at the end of the quater. Thought it could
#be useful to add in that change over time in addition to our traditional income measures

#niq: Net income total, total amount of PROFIT made in the quarter (in millions). This marks how much money the company
#managed to gain and take home after all expenses like labor, materials, taxes, etc. I like this variable because it
#might show dips in profits during the covid era, or maybe none at all because of layoffs

#revtq: revenue total, total amount of SALES made in the quarter (in millions). This marks how much money the company
#managed to gain and take home before all expenses. This is likely our most useful indicator

#gsector: NYSE sector the company belongs to. A short number identifying the sector. All sectors are grouped together
#in separate unique dataframes
discretionarydata=pd.read_csv('Consumerdiscretionary.csv')
discretionarydata1=pd.DataFrame(discretionarydata)
discretionclean=discretionarydata1.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(discretionclean)

discretionarydata2=pd.read_csv('Consumerstaples.csv')
discretionarydata3=pd.DataFrame(discretionarydata2)
consumerclean=discretionarydata3.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(consumerclean)

discretionarydata4=pd.read_csv('Industrials.csv')
discretionarydata5=pd.DataFrame(discretionarydata4)
industrialsclean=discretionarydata5.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(industrialsclean)

discretionarydata6=pd.read_csv('Energy.csv')
discretionarydata7=pd.DataFrame(discretionarydata6)
energyclean=discretionarydata7.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(energyclean)

discretionarydata8=pd.read_csv('Healthcare.csv')
discretionarydata9=pd.DataFrame(discretionarydata8)
healthcareclean=discretionarydata9.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(healthcareclean)

discretionarydata10=pd.read_csv('Financials.csv')
discretionarydata11=pd.DataFrame(discretionarydata10)
financialsclean=discretionarydata11.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(financialsclean)

discretionarydata12=pd.read_csv('Communicationservices.csv')
discretionarydata13=pd.DataFrame(discretionarydata12)
communicationclean=discretionarydata13.drop(columns=['fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc', 'datafmt', 'curcdq', 'datacqtr', 'datafqtr', 'costat','gsubind'])
#print(communicationclean)

megaset=pd.concat([discretionclean, consumerclean, industrialsclean, energyclean, healthcareclean, financialsclean, communicationclean])

part1=megaset.loc[megaset['datadate'] == '3/31/2019']
part2=megaset.loc[megaset['datadate'] == '6/30/2019']
part3=megaset.loc[megaset['datadate'] == '9/30/2019']
part4=megaset.loc[megaset['datadate'] == '12/31/2019']
test_data2=pd.concat([part1,part2,part3,part4])
test_data1=test_data2.drop(['GVKEY'], axis=1)
test_data=test_data1.set_index('datadate')
#print(test_data)
trainx=megaset.set_index('datadate')
train_data1=trainx.drop(['3/31/2019', '6/30/2019', '9/30/2019', '12/31/2019'], axis=0)
train_data=train_data1.drop(['GVKEY'], axis=1)
dates=train_data.index
#print(train_data)
#print(megaset)


X=train_data.iloc[:,1:3]
#X_train=X.as_matrix()
#print(X)
Y=train_data.iloc[:,3]
#y_train=Y.as_matrix()
#print(Y)
#X_test=test_data.iloc[:,1:3]
#print(test_x)
#y_test=test_data.iloc[:,3]
#print(test_y)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)
X_num_columns= len(X)
model = Sequential()

model.add(Dense(300,
                activation='relu'
                ))

model.add(Dense(150,
                activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(30,
                activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(7,
                activation='relu'))
model.add(Dropout(0.6))

model.add(Dense(1,
                activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(X_train, y_train, epochs=100, batch_size=128)

print(model.metrics_names)

loss = model.evaluate(X_test, y_test)

print(np.sqrt(loss))

train_data['revtq'].std()

predictions = model.predict(X)

predictions_list = map(lambda x: x[0], predictions)

predictions_series = pd.Series(predictions_list,index=dates)

dates_series =  pd.Series(dates)


for x in predictions:
    print(x[0])

Predicted_sales = model.predict(X_test)
new_dates_series=pd.Series(X_test.index)
new_predictions_list = map(lambda x: x[0], Predicted_sales)
new_predictions_series = pd.Series(new_predictions_list,index=new_dates_series)

SyntaxError: invalid syntax (<ipython-input-5-7488ac4a164b>, line 1)